In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [5]:
df = pd.read_csv('garmin_data_clean_level_1_v4.csv')

In [6]:
df['timestamp'] = df['timestamp'].apply(pd.to_datetime)

In [7]:
df = df[df['activity_id'] >=31689725151]

In [8]:
df = df[df['right_power_phase_1'].notna()]

In [9]:
df = df[df['left_power_phase_1'].notna()]

In [10]:
df = df[df['left_right_balance'].notna()]

In [11]:
df = df[df['left_torque_effectiveness'].notna()]

In [12]:
df = df[df['left_pco'].notna()]

In [13]:
df = df[df['left_pedal_smoothness'].notna()]

In [14]:
df = df.drop(['unknown_90','accumulated_power'], 1)

In [15]:
df = df.drop(['message_type','message_frame_type', 'message_global_mesg_num','message_local_mesg_num', 'message_name'], 1)

In [16]:
df = df.drop(['unknown_88','position_lat', 'position_long','enhanced_altitude','enhanced_speed'], 1)

In [17]:
df = df.drop(['fractional_cadence', 'record_id','activity_id','Unnamed: 0'], 1)

In [18]:
df['left_power_phase_length'] = df['left_power_phase_2'] - df['left_power_phase_1']

In [19]:
df.shape

(564223, 25)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 564223 entries, 14257 to 1616868
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   _id                         564223 non-null  object             
 1   timestamp                   564223 non-null  datetime64[ns, UTC]
 2   distance                    564223 non-null  float64            
 3   altitude                    564223 non-null  float64            
 4   speed                       564223 non-null  float64            
 5   power                       564223 non-null  float64            
 6   heart_rate                  564223 non-null  float64            
 7   cadence                     564223 non-null  float64            
 8   temperature                 564223 non-null  float64            
 9   left_pco                    564223 non-null  float64            
 10  right_pco                   564223 non-

In [21]:
df.corr()


,distance,altitude,speed,power,heart_rate,cadence,temperature,left_pco,right_pco,left_power_phase_1,...,right_power_phase_1,right_power_phase_2,right_power_phase_peak_1,right_power_phase_peak_2,left_right_balance,left_torque_effectiveness,right_torque_effectiveness,left_pedal_smoothness,right_pedal_smoothness,left_power_phase_length
distance,1.000000,-0.200159,-0.038026,-0.086694,0.144784,-0.064139,0.196310,0.364051,0.119099,-0.027675,...,-0.043265,-0.003173,0.045041,0.014332,0.060839,-0.068605,-0.013204,-0.135779,-0.025793,0.018167
altitude,-0.200159,1.000000,-0.136182,0.035233,0.059294,-0.056758,-0.045150,-0.059003,-0.002610,0.083270,...,0.072145,0.047449,-0.016117,0.020798,-0.038813,0.072627,0.045031,0.107581,0.043254,-0.076840
speed,-0.038026,-0.136182,1.000000,-0.022263,0.050155,0.438397,0.053343,-0.051333,-0.082924,-0.221800,...,-0.190613,-0.015474,0.085762,0.013966,-0.057100,-0.219824,-0.248638,-0.252509,-0.243899,0.225207
power,-0.086694,0.035233,-0.022263,1.000000,0.105900,-0.067764,-0.048464,-0.068341,0.166661,0.332121,...,0.343470,0.146626,0.142092,-0.120570,-0.047616,0.716513,0.654277,0.574050,0.488657,-0.313558
heart_rate,0.144784,0.059294,0.050155,0.105900,1.000000,0.195751,0.093777,0.198348,0.101428,0.049506,...,0.065599,0.039274,0.038513,-0.070973,0.035594,0.042653,0.063854,0.001820,0.007168,-0.047526
cadence,-0.064139,-0.056758,0.438397,-0.067764,0.195751,1.000000,-0.057124,0.009855,-0.154262,-0.154088,...,-0.105771,-0.096570,-0.076356,-0.114007,-0.073877,-0.270118,-0.309034,-0.222316,-0.303931,0.155174
temperature,0.196310,-0.045150,0.053343,-0.048464,0.093777,-0.057124,1.000000,-0.043677,0.030652,0.022879,...,0.011253,-0.004437,0.000066,0.020081,0.135877,-0.073699,0.010011,-0.092972,0.008328,-0.033485
left_pco,0.364051,-0.059003,-0.051333,-0.068341,0.198348,0.009855,-0.043677,1.000000,0.245839,-0.000889,...,-0.023633,0.015375,-0.054165,-0.002935,0.092103,-0.071320,0.008375,-0.115188,-0.009130,-0.006976
right_pco,0.119099,-0.002610,-0.082924,0.166661,0.101428,-0.154262,0.030652,0.245839,1.000000,0.144544,...,0.080354,0.128954,0.018753,0.002215,0.033610,0.210107,0.248938,0.160610,0.198953,-0.142498
left_power_phase_1,-0.027675,0.083270,-0.221800,0.332121,0.049506,-0.154088,0.022879,-0.000889,0.144544,1.000000,...,0.365833,0.054211,-0.030253,-0.059581,-0.045316,0.401672,0.354660,0.402519,0.293635,-0.991847


In [22]:
X = pd.DataFrame(df[['left_torque_effectiveness','right_torque_effectiveness']])
y = pd.DataFrame(df['power'])
model = LinearRegression()
scores = []
kfold = KFold(n_splits=4, shuffle=True, random_state=10)
for i, (train, test) in enumerate(kfold.split(X, y)):
 model.fit(X.iloc[train,:], y.iloc[train,:])
 score = model.score(X.iloc[test,:], y.iloc[test,:])
 scores.append(score)
print(scores)

[0.549677615826798, 0.5584841259143487, 0.5698608393815385, 0.5739425539638103]


<strong> Explaination: </strong>  This analysis has provided very interesting resutls.  Left and Right Torque Effectiveness have very high correlation with power.  Torque effectiveness measures how much of the power delivered to the pedal is pushing it forward. It is calculated as (P+ + P−) / P+ and is normally displayed as a percentage (P− will be a negative value or zero). A value of 100% means that all of the power was pushing the pedal in a positive direction (therefore, P− was zero). Values of 60–100% are common.  My values are 62% and 59% for left and right respectively which are on the low side of normal.  Effectively this states that I could increase my power output signficantly by pedalling throughout the stroke and eliminating the negative effects of working against myself. With the stats below, 25% of the time I'm cancelling out 50% of the effort I'm putting into cycling. 

In [23]:
df[['left_torque_effectiveness','right_torque_effectiveness']].describe()


,left_torque_effectiveness,right_torque_effectiveness
count,564223.000000,564223.000000
mean,61.848507,58.915260
std,12.839527,15.045813
min,0.000000,0.000000
25%,53.500000,49.500000
50%,61.000000,57.500000
75%,69.500000,67.500000
max,100.000000,100.000000
